# import the libraries

In [1]:
import os
import sys

import numpy as np
from tqdm import tqdm

from music21 import midi, converter, instrument, note, chord, stream

import tensorflow as tf
from tensorflow import keras as k
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint, LambdaCallback

import matplotlib.pyplot as plt

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


# file path
*file_path = './maestro-v3.0.0-midi/maestro-v3.0.0/'*

In [2]:
file_path = './maestro-v3.0.0'
# years = ['2014', '2015', '2017', '2018']
years = ['2014']

# list midi
print all the midi files

In [7]:
# print files inside the folder maestro-v3.0.0 with name 2004, 2006, 2008, 2009
def list_midi(file_path):
    for year in years:
        path = os.path.join(file_path, year)
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(".midi"):
                    print(os.path.join(root, file))

list_midi(file_path)

# import glob
# def list_midi_glob(file_path):
#     for file in glob.glob(os.path.join(file_path, '**/*.midi'), recursive=True):
#         print(file)

# list_midi_glob(file_path)

./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--1.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--2.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--3.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--5.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--1.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--2.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--4.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--5.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--2.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--3.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--4.midi
./maestro-v3.0.0\2014\MIDI-UNPROCESSED_01-0

# play a midi file

In [6]:
# play midi using pretty_midi fluidsynth method

def play_music(m):
    midi_stream = midi.MidiFile()
    midi_stream.open(m)
    midi_stream.read()
    midi_stream.close()
    s = midi.translate.midiFileToStream(midi_stream)
    s.show('midi')
    # display(Audio(data=midi, rate=44100))
    
play_music(file_path + '2004/MIDI-Unprocessed_XP_16_R2_2004_01_ORIG_MID--AUDIO_16_R2_2004_02_Track02_wav.midi')

FileNotFoundError: [Errno 2] No such file or directory: './maestro-v3.0.02004/MIDI-Unprocessed_XP_16_R2_2004_01_ORIG_MID--AUDIO_16_R2_2004_02_Track02_wav.midi'

# converting all the midi to stream object
- We start by loading each file into a Music21 stream object using the converter.parse(file) function.

- Using this stream object we get a list of all the notes and chords in the file.


In [8]:
def load_midi(file_path):
    all_midis_stream = []
    for year in years:
        path = os.path.join(file_path, year)
        print("Loading midi files for year: ", year)
        for root, dirs, files in os.walk(path):
            for file in tqdm(files):
                if file.endswith(".midi"):
                    tr = os.path.join(root, file)
                    # print(tr)
                    midi = converter.parse(tr)
                    all_midis_stream.append(midi)
        print("Done loading midi files for year: ", year)
    print("Done loading midi files for all years")
    return all_midis_stream


# get the notes and durations
- We append the pitch of every note object using its string notation since the most significant parts of the note can be recreated using the string notation of the pitch.

- And we append every chord by encoding the id of every note in the chord together into a single string, with each note being separated by a dot.

- These encodings allows us to easily decode the output generated by the network into the correct notes and chords.


In [9]:
def get_notes(midis_stream):
    notes = []
    durations = []
    notes_to_parse = []

    for song in tqdm(midis_stream):

        # group by instrument parts and flatten into a single stream
        part = instrument.partitionByInstrument(song)
        # part = song

        if part: # if parts has instrument parts
            notes_to_parse = part.parts[0].recurse() # get first instrument part
        else:
            notes_to_parse = midi.flat.notes # if no instrument parts, notes are flat
        
        # for part in part.parts:
        #     notes_to_parse = part.recurse()

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                # durations.append(element.duration.quarterLength)
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                # durations.append(element.duration.quarterLength)

    # return notes, durations
    return notes

In [10]:
all_midis = load_midi(file_path)
notes  = get_notes(all_midis)
n_vocab = len(set(notes))
print("Total notes in all the 2004 midis in the dataset:", len(notes))
# print("Total durations in all the 2004 midis in the dataset:", len(DURATION))

Loading midi files for year:  2014


100%|██████████| 105/105 [03:13<00:00,  1.85s/it]


Done loading midi files for year:  2014
Done loading midi files for all years


100%|██████████| 105/105 [02:33<00:00,  1.46s/it]

Total notes in all the 2004 midis in the dataset: 445854


In [5]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    # network_output = np_utils.to_categorical(network_output)
    network_output = tf.keras.utils.to_categorical(network_output)

    return (network_input, network_output)


In [6]:
# simple RNN model

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(SimpleRNN(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(SimpleRNN(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(SimpleRNN(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [ ]:
# def train(model, network_input, network_output, epochs):
#     """ train the neural network """
#     # create log files
#     with open('logs/loss.txt', 'w') as f:
#         pass

#     with open('logs/val_loss.txt', 'w') as f:
#         pass

#     with open('logs/acc.txt', 'w') as f:
#         pass

#     with open('logs/val_acc.txt', 'w') as f:
#         pass

#     # define callbacks for logging
#     log_callbacks = [
#         LambdaCallback(
#             on_epoch_end=lambda epoch, logs: append_log('logs/loss.txt', logs['loss']),
#             on_epoch_end=lambda epoch, logs: append_log('logs/val_loss.txt', logs['val_loss']),
#             on_epoch_end=lambda epoch, logs: append_log('logs/acc.txt', logs['accuracy']),
#             on_epoch_end=lambda epoch, logs: append_log('logs/val_acc.txt', logs['val_accuracy'])
#         )
#     ]

#     # save all weights
#     filepath = "weights/weights-improvement-{epoch:03d}-{loss:.4f}-bigger.hdf5"
#     checkpoint = ModelCheckpoint(
#         filepath, monitor='loss', 
#         verbose=0, 
#         save_best_only=True, 
#         mode='min'
#     )
#     callbacks_list = log_callbacks + [checkpoint]

#     model.fit(network_input, network_output, epochs, batch_size=128, callbacks=callbacks_list)


# def append_log(log_path, value):
#     """ append a value to a log file """
#     with open(log_path, 'a') as f:
#         f.write(str(value) + '\n')


In [7]:
def train(model, network_input, network_output, epochs):
    """ train the neural network """
    # save all weights
    filepath = "weights/weights-improvement-{epoch:03d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss', 
        verbose=0, 
        save_best_only=True, 
        mode='min'
    )

    model.fit(network_input, network_output, epochs, callbacks=[checkpoint])
    model.save('model.h5')

# def append_log(log_path, value):
#     """ append a value to a log file """
#     with open(log_path, 'a') as f:
#         f.write(str(value) + '\n')

In [8]:
all_midis_stream = load_midi(file_path)

notes  = get_notes(all_midis_stream)
print("Notes generated")
n_vocab = len(set(notes))
print("Vocab generated")

Loading midi files for year:  2014


100%|██████████| 105/105 [02:26<00:00,  1.39s/it]


Done loading midi files for year:  2014
Done loading midi files for all years


100%|██████████| 105/105 [01:52<00:00,  1.08s/it]

Notes generated
Vocab generated


In [9]:
network_in, network_out = prepare_sequences(notes, n_vocab)
print("Network input and output generated")

model = create_network(network_in, n_vocab)
print("Model created")
model.summary()

Network input and output generated
Model created
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 100, 512)          263168    
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100, 512)          524800    
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 512)               524800    
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropou

In [10]:
epochs = 10

print("Training in progress...")
train(model, network_in, network_out, epochs)
print("Training completed")

Training in progress...
    8/44576 [..............................] - ETA: 5:06:20 - loss: 7.7047

KeyboardInterrupt: 

In [ ]:
# def train_network():
#     """ train the neural network """
#     # epochs = 200
#     epochs = 5

#     all_midis_stream = load_midi(file_path)

#     notes  = get_notes(all_midis_stream)
#     print("Notes generated")
#     n_vocab = len(set(notes))
#     print("Vocab generated")

#     # print("Total notes in all the 2004 midis in the dataset:", len(notes))
#     # print("Total durations in all the 2004 midis in the dataset:", len(DURATION))

#     network_in, network_out = prepare_sequences(notes, n_vocab)
#     print("Network input and output generated")

#     model = create_network(network_in, n_vocab)
#     print("Model created")
#     model.summary()

#     print("Training in progress...")
#     train(model, network_in, network_out, epochs)
#     print("Training completed")


In [15]:
# train_network()

False

In [ ]:
# generate graphs

def generate_graphs():
    """ Generate graphs for loss and accuracy """
    loss = []
    val_loss = []
    acc = []
    val_acc = []

    with open('logs/loss.txt', 'r') as f:
        for line in f:
            loss.append(float(line.strip()))

    with open('logs/val_loss.txt', 'r') as f:
        for line in f:
            val_loss.append(float(line.strip()))

    with open('logs/acc.txt', 'r') as f:
        for line in f:
            acc.append(float(line.strip()))

    with open('logs/val_acc.txt', 'r') as f:
        for line in f:
            val_acc.append(float(line.strip()))

    plt.plot(loss)
    plt.plot(val_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'])
    plt.savefig('loss.png')

    plt.plot(acc)
    plt.plot(val_acc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'])
    plt.savefig('accuracy.png')

generate_graphs()